In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
from gensim.models import Word2Vec
import datetime
from dateutil.relativedelta import relativedelta
from collections import Counter
from sklearn.decomposition import PCA
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances
from sklearn.feature_extraction.text import CountVectorizer

C:\Users\Lassana Diabira\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
train=pd.read_csv("Training_set.csv")

In [3]:
train=train.sample(20000)

In [4]:
test=pd.read_csv("Test_set.csv")

In [5]:
def get_job_characteristics(df): 
    job_type=[]
    job_title=[]
    employer_title=[]
    employer_sector_title=[]
    job_sector_title=[]
    for i in range(df.shape[0]):
        data=df.iloc[i]
        types=data["job_type"]
        title=data["job_title"]
        em_title=data["employer_title"]
        em_sec_title=data["employer_sector_title"]
        job_sec_title=data["job_sector_title"]
        job_type.append(types)
        job_title.append(title)
        employer_title.append(em_title)
        employer_sector_title.append(em_sec_title)
        job_sector_title.append(job_sec_title)
    d={}
    #d["job_title"]=job_title
    #d["employer_title"]=employer_title
    d["employer_sector_title"]=employer_sector_title
    d["job_sector_title"]=job_sector_title
    a=pd.DataFrame(d)
    data=pd.get_dummies(a)
    b=pd.get_dummies(job_type)
    result = pd.concat([data, b], axis=1, sort=False)
    return result

In [6]:
train.columns

Index(['Unnamed: 0', 'user_id', 'job_id', 'employer_id', 'job_opening_date',
       'job_closing_date', 'job_description', 'job_type', 'job_title',
       'employer_title', 'employer_description', 'employer_sector_id',
       'employer_sector_title', 'jobsector_id', 'job_sector_title', 'gender',
       'ethnicity', 'school_type', 'course_includes_industrial_placement',
       'graduation_year', 'right_to_work_uk', 'degree_subject_id',
       'university_id', 'university_name', 'university_type',
       'degree_subject_name', 'user_preference'],
      dtype='object')

In [7]:
def get_data_for_user(user):
    jobtitles=list(user["job_title"].values)
    emtitles=list(user["employer_title"].values)
    jobsectitles=list(user["job_sector_title"].values)
    emsectitles=list(user["employer_sector_title"].values)
    a=""
    for k in range(len(jobtitles)):
        a+=jobtitles[k]+" "
        a+=emtitles[k]+" "
        a+=jobsectitles[k]+" "
        #a+=emsectitles[k]+" "
    return a

In [8]:
def get_user_characteristics(df):
    corpus=[]
    gender=[]
    ethnicity=[]
    school_type=[]
    right_to_work_uk=[]
    university_name=[]
    university_type=[]
    course_includes_industrial_placement=[]
    degree_subject_name=[]
    user_preference=[]
    job_types=list(set(df["job_type"].values))
    em_sectors=list(set(df["employer_sector_title"].values))
    job_sectors=list(set(df["job_sector_title"].values))
    count_job_types=[[] for i in range(len(job_types))]
    count_job_sectors=[[] for i in range(len(job_sectors))]
    count_em_sectors=[[] for i in range(len(em_sectors))]
    nb_em_sector=[]
    nb_job_sector=[]
    job_type=[]
    for i in range(df.shape[0]):
        data=df.iloc[i]
        user=data["user_id"]
        user_data=df[df["user_id"]==user]
        user_job_type=dict(Counter(list(user_data["job_type"].values)))
        user_job_sectors=dict(Counter(list(user_data["job_sector_title"].values)))
        user_em_sectors=dict(Counter(list(user_data["employer_sector_title"].values)))
        user_em_sector=len(list(set(list(user_data["employer_sector_title"].values))))
        user_job_sector=len(list(set(list(user_data["job_sector_title"].values))))
        for i in range(len(count_job_types)):
            try:
                count_job_types[i].append(user_job_type[i])
            except:
                count_job_types[i].append(0)
        for i in range(len(count_job_sectors)):
            try:
                count_job_sectors[i].append(user_job_sectors[i])
            except:
                count_job_sectors[i].append(0)
        for i in range(len(count_em_sectors)):
            try:
                count_em_sectors[i].append(user_em_sectors[i])
            except:
                count_em_sectors[i].append(0)
        sex=data["gender"]
        jobtype=data["job_type"]
        eth=data["ethnicity"]
        school=data["school_type"]
        placement=data["course_includes_industrial_placement"]
        right=data["right_to_work_uk"]
        uni=data["university_name"]
        uni_type=data["university_type"]
        subname=data["degree_subject_name"]
        pref=data["user_preference"]
        title=data["job_title"]
        em_title=data["employer_title"]
        em_sec_title=data["employer_sector_title"]
        job_sec_title=data["job_sector_title"]
        if sex=="F":
            sex="Female"
        elif sex=="M":
            sex="Male"
        if placement==True:
            placement=1
        elif placement==False:
            placement=-1
        else:
            placement=0
        if right==True:
            right=1
        elif right==False:
            right=-1
        else:
            right=0
        if type(pref) is float:
            pref=""
        if type(subname) is float:
            subname=""
        nb_em_sector.append(user_em_sector)
        nb_job_sector.append(user_job_sector)
        gender.append(sex)
        ethnicity.append(eth)
        school_type.append(school)
        course_includes_industrial_placement.append(placement)
        right_to_work_uk.append(right)
        university_type.append(uni_type)
        #degree_subject_name.append(subname)
        #user_preference.append(pref)
        job_type.append(jobtype)
        corpus.append(pref+subname+" "+get_data_for_user(user_data))
    d={}
    for i in range(len(count_job_sectors)):
        d['count_job_sectors_'+str(i)]=count_job_sectors[i]
    for i in range(len(count_em_sectors)):
        d['count_em_sectors_'+str(i)]=count_em_sectors[i]
    for i in range(len(count_job_types)):
        d['count_job_types_'+str(i)]=count_job_types[i]
    d["gender"]=gender
    d["job_type"]=job_type
    d["nb_em_sector"]=nb_em_sector
    d["nb_job_sector"]=nb_job_sector
    d["right_to_work_uk"]=right_to_work_uk
    d["ethnicity"]=ethnicity
    #d["degree_subject_name"]=degree_subject_name
    d["school_type"]=school_type
    #d["user_preference"]=user_preference
    d["course_includes_industrial_placement"]=course_includes_industrial_placement
    a=pd.DataFrame(d)
    data=pd.get_dummies(a)
    b=pd.get_dummies(university_type)
    result = pd.concat([data, b], axis=1, sort=False)
    vectorizer = CountVectorizer(max_df=1.0, min_df=3)
    X = vectorizer.fit_transform(corpus)
    col=vectorizer.get_feature_names()
    donnees=X.toarray()
    print(X.shape)
    donnee=pd.DataFrame(donnees, columns=col)
    final=pd.concat([result, donnee], axis=1)
    return final

In [9]:
user_characteristics=get_user_characteristics(train)

(20000, 1510)


In [10]:
user_characteristics.shape

(20000, 1594)

In [10]:
users=list(train["user_id"].values)
users=[str(i) for i in users]

In [11]:
pca = PCA(n_components=2)
test_user = pca.fit_transform(user_characteristics)

In [12]:
print(pca.explained_variance_ratio_)

[0.99385504 0.00205412]


In [ ]:
k=5500
f=7000
plt.figure(figsize=(20,20))
plt.scatter(test_user[k:f, 0], test_user[k:f, 1])
for i in range(k,f):
    plt.annotate(users[i], xy=(test_user[i, 0], test_user[i, 1]))
plt.show()

In [14]:
def select_sector_left(df, x, n, treshold):
    res=[]
    a=0
    b=0
    while a<20000 and b==0:
        if x[a,0]<treshold:
            user=df.iloc[a]["user_id"]
            if user not in res:
                res.append(user)
        a+=1
        if len(res)>=n:
            b=1
    return res

def select_sector_right(df, x, n, treshold):
    res=[]
    a=0
    b=0
    while a<10000 and b==0:
        if x[a,0]>treshold:
            user=df.iloc[a]["user_id"]
            if user not in res:
                res.append(user)
        a+=1
        if len(res)>=n:
            b=1
    return res

def select_sector_between(df, x, n, treshold_left, treshold_right):
    res=[]
    a=0
    b=0
    while a<10000 and b==0:
        if x[a,0]>treshold_left and x[a,0]<treshold_right:
            user=df.iloc[a]["user_id"]
            if user not in res:
                res.append(user)
        a+=1
        if len(res)>=n:
            b=1
    return res

In [15]:
left=select_sector_left(train, test_user, 5, -120)

In [16]:
left

[]

In [17]:
right=select_sector_right(train, test_user, 5, 30)

In [18]:
right

[164402, 67665, 36114]

In [19]:
between=select_sector_between(train, test_user, 5, -100, -80)

In [20]:
between

[98304, 65537, 65540, 65541, 65546]

In [11]:
job_characteristics=get_job_characteristics(train)

In [12]:
job_characteristics.shape

(20000, 57)

In [13]:
def get_user_info(df):
    user_info=[]
    x=train[["user_id","job_title","employer_title","job_sector_title","job_description",
             "employer_sector_title","employer_description"]]
    n=df.shape[0]
    for i in range(n):
        if i%5000==0:
            print("progress: ",i/n)
        data=df.iloc[i]
        user_id=data["user_id"]
        y=x[x["user_id"]==user_id]
        job_titles=y["job_title"].values
        job_sector_title=y["job_sector_title"].values
        description=y["job_description"].values
        em_titles=y["employer_title"].values
        em_sector_title=y["employer_sector_title"].values
        em_description=y["employer_description"].values
        university_name=data["university_name"]
        degree_subject_name=data["degree_subject_name"]
        user_preference=data["user_preference"]
        ethnicity=data["ethnicity"]
        school_type=data["school_type"]
        if type(user_preference) is float:
            user_preference=""
        if type(university_name) is float:
            university_name=""
        if type(degree_subject_name) is float:
            degree_subject_name=""
        if type(ethnicity) is float:
            ethnicity=""
        if type(school_type) is float:
            school_type=""
        user_preference=user_preference+" "
        user_preference=user_preference *3
        degree_subject_name=degree_subject_name+" "
        degree_subject_name=degree_subject_name *3
        a=""
        a+=university_name+" "
        a+=degree_subject_name+" "
        a+=user_preference+" "
        a+=ethnicity+" "
        a+=school_type+" "
        for k in range(len(job_titles)):
            a+=job_titles[k]+" "
            a+=job_sector_title[k]+" "
            a+=description[k]+" "
            a+=em_titles[k]+" "
            a+=em_sector_title[k]+" "
            a+=em_description[k]+" "
        a=a.replace("Graduate","").replace("Programme","").replace("  "," ").replace("&","")
        user_info.append(a)
    return user_info

In [14]:
train_user_info=get_user_info(train)

progress:  0.0
progress:  0.25
progress:  0.5
progress:  0.75


In [15]:
train_user_info[15]

'London Southbank University Engineering - Mechanical Engineering - Mechanical Engineering - Mechanical Commercial Law Investment Banking Financial Services Commercial Law Investment Banking Financial Services Commercial Law Investment Banking Financial Services  Black/African/Carribean/Black British Private Open Days London/Bristol 2017/18 Commercial Law <p>We will be hosting a series ofnbsp;Open Days during 2017/18 in our London office:</p>\r\r\n\r\r\n<ul>\r\r\n\t<li>04 October</li>\r\r\n\t<li>18 October</li>\r\r\n\t<li>21nbsp;March</li>\r\r\n\t<li>27 June</li>\r\r\n</ul>\r\r\n\r\r\n<p>We will alsonbsp;be hostingnbsp;Open Days during 2017 in our Bristol office:</p>\r\r\n\r\r\n<ul>\r\r\n\t<li>30nbsp;October AM</li>\r\r\n\t<li>30nbsp;October PM</li>\r\r\n\t<li>29nbsp;November AM</li>\r\r\n\t<li>29nbsp;November PM</li>\r\r\n</ul>\r\r\n\r\r\n<p>Applications are open to anyone of undergraduate level or above.</p> Simmons  Simmons Commercial Law <p>Scratch the surface of Simmons amp; Simmo

In [23]:
def clean_user(x):
    n=len(x)
    res=[] 
    for i,content in enumerate(x):
        if i%5000==0:
            print("progress : ",i/n)
        content=content.replace(",","").replace("-"," ").replace("  "," ")
        content=content.replace("(","").replace(")","").replace(" - "," ").replace(" / "," ").replace("/"," ").replace("   ","")
        content=content.replace("&bull;","\r\n ").replace("bull;","\r\n ").replace("\r\r\n\t"," ").replace("\r\r\n"," ")
        content=content.replace("\r\r\n\r\r\n"," ").replace("..."," ").replace("&#39;s","").replace(":","")
        content=content.replace("&eacute;","e").replace("&lsquo;","").replace("lsquo;","").replace("&#39;","").replace("#39;s","")
        content=content.replace("&nbsp;"," ").replace("nbsp;"," ").replace("&#39;","").replace("amp;","").replace("&middot;","")
        content=content.replace("."," ").replace(",","").replace("?"," ").replace("!","").replace("  "," ").replace("middot;","")
        content=content.replace("&rsquo;","").replace("rsquo;"," ").replace("\t"," ").replace("\r\n\r\n","").replace(" \r\n ","")
        content=content.replace("&ndash;"," ").replace("ndash;"," ").replace("\r\n\r\n"," ").replace(" \r\n "," ").replace("\r\n"," ")
        content=content.replace("\r"," ").replace("  ","").replace("   ","").replace("  ","").replace("   ","")
        content=content.replace("(","").replace(")","").replace(" - "," ").replace(" / "," ").replace("/"," ").replace("   ","")
        content=re.sub('<.*?>',"",content , flags=re.DOTALL)
        content=content.split(" ")
        y=[]
        for i in content:
            if len(i)>1:
                y.append(i.lower())
        res.append(y)
    return res

In [24]:
clean_train_user_info=clean_user(train_user_info)

progress :  0.0
progress :  0.25
progress :  0.5
progress :  0.75


In [25]:
clean_train_user_info[15]

['london',
 'southbank',
 'university',
 'engineering',
 'mechanical',
 'engineering',
 'mechanical',
 'engineering',
 'mechanical',
 'commercial',
 'law',
 'investment',
 'banking',
 'financial',
 'services',
 'commercial',
 'law',
 'investment',
 'banking',
 'financial',
 'services',
 'commercial',
 'law',
 'investment',
 'banking',
 'financial',
 'services',
 'black',
 'african',
 'carribean',
 'black',
 'british',
 'private',
 'open',
 'days',
 'london',
 'bristol',
 '2017',
 '18',
 'commercial',
 'law',
 'we',
 'will',
 'be',
 'hosting',
 'series',
 'ofnbsp;open',
 'days',
 'during',
 '2017',
 '18',
 'in',
 'our',
 'london',
 'office',
 '04',
 'october',
 '18',
 'october',
 '21nbsp;march',
 '27',
 'june',
 'we',
 'will',
 'alsonbsp;be',
 'hostingnbsp;open',
 'days',
 'during',
 '2017',
 'in',
 'our',
 'bristol',
 'office',
 '30nbsp;october',
 'am',
 '30nbsp;october',
 'pm',
 '29nbsp;november',
 'am',
 '29nbsp;november',
 'pm',
 'applications',
 'are',
 'open',
 'to',
 'anyone',
 '

In [19]:
def get_job_info(df):
    job_info=[]
    for i in range(df.shape[0]):
        data=df.iloc[i]
        title=data["job_title"]
        job_sector_title=data["job_sector_title"]
        description=data["job_description"]
        em_title=data["employer_title"]
        em_sector_title=data["employer_sector_title"]
        em_description=data["employer_description"]
        a=""
        a+=title+" "
        a+=job_sector_title+" "
        a+=description+" "
        a+=em_title+" "
        a+=em_sector_title+" "
        a+=em_description
        job_info.append(a)
    return job_info

In [20]:
train_job_info=get_job_info(train)

In [21]:
def clean(x):
    res=[] 
    for content in x:
        content=content.replace(",","").replace("-"," ").replace("  "," ")
        content=content.replace("(","").replace(")","").replace(" - "," ").replace(" / "," ").replace("/"," ").replace("   ","")
        content=content.replace("&bull;","\r\n ").replace("bull;","\r\n ").replace("\r\r\n\t"," ").replace("\r\r\n"," ")
        content=content.replace("\r\r\n\r\r\n"," ").replace("..."," ").replace("&#39;s","").replace(":","")
        content=content.replace("&eacute;","e").replace("&lsquo;","").replace("lsquo;","").replace("&#39;","").replace("#39;s","")
        content=content.replace("&nbsp;"," ").replace("nbsp;"," ").replace("&#39;","").replace("amp;","").replace("&middot;","")
        content=content.replace("."," ").replace(",","").replace("?"," ").replace("!","").replace("  "," ").replace("middot;","")
        content=content.replace("&rsquo;","").replace("rsquo;"," ").replace("\t"," ").replace("\r\n\r\n","").replace(" \r\n ","")
        content=content.replace("&ndash;"," ").replace("ndash;"," ").replace("\r\n\r\n"," ").replace(" \r\n "," ").replace("\r\n"," ")
        content=content.replace("\r"," ").replace("  ","").replace("   ","").replace("  ","").replace("   ","")
        content=content.replace("(","").replace(")","").replace(" - "," ").replace(" / "," ").replace("/"," ").replace("   ","")
        content=re.sub('<.*?>',"",content , flags=re.DOTALL)
        content=content.split(" ")
        y=[]
        for i in content:
            if len(i)>1:
                y.append(i.lower())
        res.append(y)
    return res

In [22]:
cleaned_train_job_info=clean(train_job_info)

In [26]:
def remove_stopwords(x):
    res=[]
    with open('stopwords.txt') as stopfile:
        stopwords = stopfile.read()
        stop = stopwords.split()
    for i,job in enumerate(x):
        a=[]
        for word in job:
            if word not in stop:
                a.append(word)
        res.append(a)
    return res

In [27]:
train_clean=remove_stopwords(cleaned_train_job_info)

In [28]:
train_user_clean=remove_stopwords(clean_train_user_info)

In [29]:
def get_occurence(x):
    res=[]
    for job in x:
        res.append(dict(Counter(job)))
    return res

In [30]:
occurences=get_occurence(train_clean)

In [31]:
train_user_occurences=get_occurence(train_user_clean)

In [32]:
def get_vector(occurences, jobs, vector_size):
    vector=[]
    model = Word2Vec.load('model.bin')
    nb=len(jobs)
    for i,job in enumerate(jobs):
        if i%10000==0:
            print("progress : ",i/nb )
        res=np.zeros(vector_size,)
        n=sum(list(occurences[i].values()))
        job_unique=list(set(job))
        for word in job_unique:
            try:
                a=model[word]
            except:
                print(word)
                a=None
            if a is not None:
                count=occurences[i][word]
                w=count/n
                res+=w*a
        vector.append(res)
    return vector

In [ ]:
X=get_vector(occurences, train_clean, 200)

In [ ]:
X_user=get_vector(train_user_occurences, train_user_clean, 200)

In [35]:
X_train=np.array(X)

In [36]:
X_train.shape

(20000, 200)

In [37]:
X_train_user=np.array(X_user)

In [38]:
X_train_user.shape

(20000, 200)

In [39]:
X_train_job_final=np.concatenate((X_train, job_characteristics.values), axis=1)

In [40]:
X_train_job_final.shape

(20000, 257)

In [41]:
X_train_user_final=np.concatenate((X_train_user, user_characteristics.values), axis=1)

In [42]:
X_train_user_final.shape

(20000, 1794)

In [43]:
pca = PCA(n_components=2)
result_user = pca.fit_transform(X_train_user_final)

In [44]:
print(pca.explained_variance_ratio_)

[0.99129471 0.00174109]


In [45]:
pca = PCA(n_components=2)
result = pca.fit_transform(X_train_job_final)

In [46]:
print(pca.explained_variance_ratio_)

[0.15698146 0.11857781]


In [58]:
users=list(train["user_id"].values)
users=[str(i) for i in users]

In [59]:
def get_job_info_plot(df):
    job_info=[]
    for i in range(df.shape[0]):
        data=df.iloc[i]
        title=data["job_title"]
        em_title=data["employer_title"]
        a=""
        a+=title+" "
        a+=em_title+" "
        job_info.append(a)
    return job_info

In [60]:
infos=get_job_info_plot(train)

In [ ]:
k=200
plt.figure(figsize=(20,20))
plt.scatter(result_user[:k, 0], result_user[:k, 1])
for i, word in enumerate(users[:k]):
    plt.annotate(word, xy=(result_user[i, 0], result_user[i, 1]))
plt.show()

In [63]:
train.head()

,Unnamed: 0,user_id,job_id,employer_id,job_opening_date,job_closing_date,job_description,job_type,job_title,employer_title,...,school_type,course_includes_industrial_placement,graduation_year,right_to_work_uk,degree_subject_id,university_id,university_name,university_type,degree_subject_name,user_preference
89566,89566,154806,4208.0,386.0,2018-01-18,2018-02-09,<p>&nbsp;</p>\r\r\n\r\r\n<p>&nbsp;</p>\r\r\n\r...,3.0,Finance Summer Intern - London,Expedia,...,State,False,2019.0,True,2.0,436.0,Warwick Business School,2.0,Accounting & Finance,NaN
57119,57119,182299,4147.0,324.0,2017-12-15,2018-02-23,"<p>Born in 1981, Bloomberg is the world&#39;s ...",3.0,Summer Internship 2018: Government Relations,Bloomberg,...,Private,True,2018.0,True,1396.0,474.0,Loughborough Univeristy,2.0,Sport Science with Management,Investment Banking Investment & Asset Manageme...
107684,107684,126087,2679.0,30.0,2017-09-06,2018-01-14,<h2>Nomura overview</h2>\r\r\n\r\r\n<p>Nomura ...,3.0,2018 Women's Immersion Programme - London,Nomura,...,NaN,False,2020.0,True,254.0,337.0,University of Bristol,2.0,Chemistry,NaN
122112,122112,188647,2830.0,25.0,2017-09-12,NaN,<h2>Graduate Programme</h2>\r\r\n\r\r\n<h3>Fin...,1.0,Graduate Programme,Deloitte,...,Private,False,2018.0,True,1147.0,186.0,City University London,2.0,Msc Organisational Psychology,HR & Recruitment
73812,73812,152385,1552.0,106.0,2017-09-29,2018-04-22,<h2>Discover the difference you can make in an...,3.0,Risk Internship Programme,Lloyds Banking Group,...,NaN,False,NaN,False,NaN,NaN,NaN,NaN,NaN,HR & Recruitment Investment Banking Management...


In [52]:
def get_similar_member(df, X_train, doc_id, n):
    res=[]
    beta=[]
    cos=list(cosine_similarity(X_train[doc_id,:].reshape(1, -1) , X_train))[0]
    most_similar_doc_id=np.argsort(cos)[::-1]
    a=0
    while cos[most_similar_doc_id[a]]>=0.9999999:
        a+=1
    most_similar_doc_id=most_similar_doc_id[a:]
    res.append(df.iloc[most_similar_doc_id[0]]["user_id"])
    beta.append(cos[most_similar_doc_id[0]])
    for i in range(1,len(most_similar_doc_id)):
        if cos[most_similar_doc_id[i]] not in beta:
            user=df.iloc[most_similar_doc_id[i]]["user_id"]
            if user not in res:
                res.append(user)
                beta.append(cos[most_similar_doc_id[i]])
    print(" user id : ", df.iloc[doc_id]["user_id"])
    print("most similar doc : ", res[:n])
    print("most similar values : ", beta[:n])
    print("not similar doc : ", res[-n:][::-1])
    print("not similar values : ", beta[-n:][::-1])

In [79]:
get_similar_member(train, X_train_user, 2, 5)

 user id :  126087
most similar doc :  [120628, 83192, 137495, 155295, 184529]
most similar values :  [0.966291342212857, 0.9608692779860268, 0.9596474524662439, 0.9564854309323045, 0.9550827148909369]
not similar doc :  [152468, 194398, 98253, 74505, 129195]
not similar values :  [-0.19019082597602052, -0.1579968032257406, 0.00832261069361534, 0.0170742763480088, 0.02207867162846998]


# user

In [65]:
train[train["user_id"]==126087][['job_title','employer_title', 'employer_sector_title', 'job_sector_title', 'gender',
       'ethnicity', 'university_name', 'degree_subject_name', 'user_preference']].tail(10)

,job_title,employer_title,employer_sector_title,job_sector_title,gender,ethnicity,university_name,degree_subject_name,user_preference
107684,2018 Women's Immersion Programme - London,Nomura,Financial Services,Financial Services,Female,Black/African/Carribean/Black British,University of Bristol,Chemistry,NaN
107676,Summer Internship Programme - Hong Kong,MUFG,Financial Services,Financial Services,Female,Black/African/Carribean/Black British,University of Bristol,Chemistry,NaN
107673,Summer Internship,Rolls-Royce,Technology: Consulting & Project Management,Technology: Consulting & Project Management,Female,Black/African/Carribean/Black British,University of Bristol,Chemistry,NaN
107668,Intern Programme 2018,Berenberg,Financial Services,Financial Services,Female,Black/African/Carribean/Black British,University of Bristol,Chemistry,NaN
107660,Investment Internship Scheme,M&G Investments,Financial Services,Financial Services,Female,Black/African/Carribean/Black British,University of Bristol,Chemistry,NaN
107656,Corporate Risk and Broking Intern,Willis Towers Watson,Financial Services,Financial Services,Female,Black/African/Carribean/Black British,University of Bristol,Chemistry,NaN


# similar users

In [80]:
train[train["user_id"]==120628][['job_title','employer_title', 'employer_sector_title', 'job_sector_title', 'gender',
       'ethnicity', 'university_name', 'degree_subject_name', 'user_preference']].tail()

,job_title,employer_title,employer_sector_title,job_sector_title,gender,ethnicity,university_name,degree_subject_name,user_preference
78908,Intern Programme 2018,Berenberg,Financial Services,Financial Services,Male,Asian/Asian British,London School of Economics and Political Scien...,Mathematics,Investment Banking Insurance & Risk Management...
78904,Business Intern,Google,Technology: Consulting & Project Management,Technology: Consulting & Project Management,Male,Asian/Asian British,London School of Economics and Political Scien...,Mathematics,Investment Banking Insurance & Risk Management...
78912,Assurance & Business Services Summer Internship,Smith & Williamson,Financial Services,"Accounting, Tax & Audit",Male,Asian/Asian British,London School of Economics and Political Scien...,Mathematics,Investment Banking Insurance & Risk Management...
78910,Proprietary Trading Intern,DE Shaw & Co.,Financial Services,Financial Services,Male,Asian/Asian British,London School of Economics and Political Scien...,Mathematics,Investment Banking Insurance & Risk Management...
78894,Summer Analyst Programme,BlackRock,Financial Services,Financial Services,Male,Asian/Asian British,London School of Economics and Political Scien...,Mathematics,Investment Banking Insurance & Risk Management...


In [67]:
train[train["user_id"]==83192][['job_title','employer_title', 'employer_sector_title', 'job_sector_title', 'gender',
       'ethnicity', 'university_name', 'degree_subject_name', 'user_preference']].tail(7)

,job_title,employer_title,employer_sector_title,job_sector_title,gender,ethnicity,university_name,degree_subject_name,user_preference
52922,Investment Graduate Scheme,M&G Investments,Financial Services,Financial Services,Female,NaN,London School of Economics and Political Scien...,Economics,NaN
52920,International Talent Programme,ING,Financial Services,Financial Services,Female,NaN,London School of Economics and Political Scien...,Economics,NaN
52924,Internship,Oliver Wyman,Consulting,Consulting,Female,NaN,London School of Economics and Political Scien...,Economics,NaN


# not similar users

In [61]:
train[train["user_id"]==152468][['job_title','employer_title', 'employer_sector_title', 'job_sector_title', 'gender',
       'ethnicity', 'university_name', 'degree_subject_name', 'user_preference']].tail(7)

,job_title,employer_title,employer_sector_title,job_sector_title,gender,ethnicity,university_name,degree_subject_name,user_preference
74184,Herbert Smith Freehills Event Dates 2017-2018,Herbert Smith Freehills,Commercial Law,Commercial Law,Male,Asian/Asian British,London School of Economics and Political Scien...,Business Mathematics and Statistics,"Commercial Law Accounting, Tax & Audit Public ..."


In [69]:
train[train["user_id"]==194398][['job_title','employer_title', 'employer_sector_title', 'job_sector_title', 'gender',
       'ethnicity', 'university_name', 'degree_subject_name', 'user_preference']].tail(7)

,job_title,employer_title,employer_sector_title,job_sector_title,gender,ethnicity,university_name,degree_subject_name,user_preference
119672,Application tester,ACME,Education & Teaching,Insurance & Risk Management,Female,White/White British,University of Exeter,Classical Studies with English,NaN


In [70]:
train[train["user_id"]==98253][['job_title','employer_title', 'employer_sector_title', 'job_sector_title', 'gender',
       'ethnicity', 'university_name', 'degree_subject_name', 'user_preference']].tail(7)

,job_title,employer_title,employer_sector_title,job_sector_title,gender,ethnicity,university_name,degree_subject_name,user_preference
125035,Campus Ambassadors,Reed Smith,Commercial Law,Commercial Law,Female,NaN,Durham University,History and Spanish,Commercial Law


In [47]:
def get_similar_doc(X_train, doc_id, n):
    res=[]
    beta=[]
    cos=list(cosine_similarity(X_train[doc_id,:].reshape(1, -1) , X_train))[0]
    most_similar_doc_id=np.argsort(cos)[::-1]
    a=0
    while cos[most_similar_doc_id[a]]>=0.9999999:
        a+=1
    most_similar_doc_id=most_similar_doc_id[a:]
    res.append(most_similar_doc_id[0])
    beta.append(cos[most_similar_doc_id[0]])
    for i in range(1,len(most_similar_doc_id)):
        if cos[most_similar_doc_id[i]] not in beta:
            res.append(most_similar_doc_id[i])
            beta.append(cos[most_similar_doc_id[i]])
    print("most similar doc : ", res[:n])
    print("most similar values : ", beta[:n])
    print("not similar doc : ", res[-n:][::-1])
    print("not similar values : ", beta[-n:][::-1])

In [73]:
def clean_similarity(x):
    res=[] 
    for content in x:
        content=content.replace(",","").replace("-"," ").replace("  "," ")
        content=content.replace("(","").replace(")","").replace(" - "," ").replace(" / "," ").replace("/"," ").replace("   ","")
        content=content.replace("&bull;","\r\n ").replace("bull;","\r\n ").replace("\r\r\n\t"," ").replace("\r\r\n"," ")
        content=content.replace("\r\r\n\r\r\n"," ").replace("..."," ").replace("&#39;s","").replace(":","")
        content=content.replace("&eacute;","e").replace("&lsquo;","").replace("lsquo;","").replace("&#39;","").replace("#39;s","")
        content=content.replace("&nbsp;"," ").replace("nbsp;"," ").replace("&#39;","").replace("amp;","").replace("&middot;","")
        content=content.replace("."," ").replace(",","").replace("?"," ").replace("!","").replace("  "," ").replace("middot;","")
        content=content.replace("&rsquo;","").replace("rsquo;"," ").replace("\t"," ").replace("\r\n\r\n","").replace(" \r\n ","")
        content=content.replace("&ndash;"," ").replace("ndash;"," ").replace("\r\n\r\n"," ").replace(" \r\n "," ").replace("\r\n"," ")
        content=content.replace("\r"," ").replace("  ","").replace("   ","").replace("  ","").replace("   ","")
        content=content.replace("(","").replace(")","").replace(" - "," ").replace(" / "," ").replace("/"," ").replace("   ","")
        content=re.sub('<.*?>',"",content , flags=re.DOTALL)
        res.append(content)
    return res

In [71]:
get_similar_doc(X_train, 5000, 5)

most similar doc :  [7941, 19818, 13407, 381, 1616]
most similar values :  [0.7986623947714357, 0.7970652822865778, 0.7961828348724959, 0.7947366931508569, 0.7932868887128695]
not similar doc :  [15493, 2708, 5869, 8481, 1089]
not similar values :  [-0.25818983588615957, -0.19147226189975503, -0.07423986285286917, -0.06118415496378256, -0.027508785071233076]


In [74]:
data_similarity=clean_similarity(train_job_info)

# job

In [75]:
data_similarity[5000]

'2018 Associate Programme Charity & Social Enterprise Overview The Associate Programme transforms your career Its a way to learn on the job doing real work in purpose led organisations while at the same time going through an intensive learning programme with a community of like minded professionals    Placements youll work in two different organisations over the course of the year and get hands on experience    Training experts in their field from diverse sectors share their knowledge and experience with you every Friday afternoon    Mentors & Coaches you meet a mentor fortnightly who guides and challenges you to add value in your placement You meet an executive coach every quarter focused on your personal development    The community we bring together our growing community though engaging events and a close social network   Who we look for   Motivation and persistence the ability to turn ideas into reality and seize opportunities to challenge yourself and set goals and knowing when to

# Similar jobs

In [89]:
data_similarity[111058]

'Graduate Resourcer Consultant Consulting Overview Robert Half is currently looking to recruit a Graduate Resourcer Consultant to join their growing Permanent Services Division in Central Nottingham This is a busy exciting role which is demanding and rewarding in equal measure  Robert Half is currently looking to recruit a Graduate Resourcer Consultant to join their growing Permanent Services Division in Central Nottingham This is a busy exciting role which is demanding and rewarding in equal measure The opportunity is massive for both earning potential and career development although consistent hard work will be required to achieve these  The Role This is a broad ranging role that will see you gain an ideal entry point into a recruitment career in one of the largest professional recruitment businesses worldwide Your key responsibilities will be  Attracting and engaging candidates to Robert Half through writing job adverts social media monitoring databases and telephone work Implement 

# not similar jobs

In [91]:
data_similarity[119665]

'Data Engineer Analytics Intern Summer 2018 Technology Consulting & Project Management Overview Facebook mission is to give people the power to build community and bring the world closer together Through our family of apps and services were building a different kind of company that connects billions of people around the world gives them ways to share what matters most to them and helps bring people closer together Whether were creating new products or helping a small business expand its reach people at Facebook are builders at heart Our global teams are constantly iterating solving problems and working together to empower people around the world to build community and connect in meaningful ways Together we can help people build stronger communities &mdash; were just getting started  Would you like to work with big data Do you want to use data to influence product decisions for products being used by over half a billion people every day If yes we want to talk to you Our data warehouse t

In [43]:
X_train.shape

(125136, 200)

In [44]:
job_characteristics.shape

(125136, 6)